# HDBSCAN should work well on embedding representations

In [1]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "RSS"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 320
10.0 percent of files read.
20.0 percent of files read.
30.0 percent of files read.
40.0 percent of files read.
50.0 percent of files read.
60.0 percent of files read.
70.0 percent of files read.
80.0 percent of files read.
90.0 percent of files read.
100.0 percent of files read.
(78416, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,Hurricane Dorian lashes US as Bahamas counts cost,"Life-threatening US storm surges are feared, a...","Thu, 05 Sep 2019 16:03:44 GMT",https://www.bbc.co.uk/news/world-us-canada-495...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Hurricane Dorian lashes US as Bahamas counts c...
1,1,Kohistan video murders: Three guilty in 'honou...,They are relatives of a group of Pakistani wom...,"Thu, 05 Sep 2019 13:53:17 GMT",https://www.bbc.co.uk/news/world-asia-49592540,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Kohistan video murders: Three guilty in 'honou...
2,2,MH17 Ukraine plane crash: 'Key witness' released,A Ukrainian court releases a potentially key w...,"Thu, 05 Sep 2019 13:46:06 GMT",https://www.bbc.co.uk/news/world-europe-49591148,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,MH17 Ukraine plane crash: 'Key witness' releas...
3,3,Article 370: The weddings 'ruined' by Kashmir'...,Indian-administered Kashmir is under a securit...,"Thu, 05 Sep 2019 07:32:34 GMT",https://www.bbc.co.uk/news/world-asia-india-49...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Article 70: The weddings 'ruined' by Kashmir's...
4,4,Syria war: Turkey warns Europe of new migrant ...,President Erdogan demands international help t...,"Thu, 05 Sep 2019 16:11:48 GMT",https://www.bbc.co.uk/news/world-europe-49599297,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Syria war: Turkey warns Europe of new migrant ...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [5]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
        for line in infile:
            outfile.write(line.encode('cp1252'))

In [117]:
infersent = reps.InferSentModel(list(corpus['clean_text']),
                                list(corpus['clean_text']),
                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

embeddings = infersent.get_embeddings()

Found 16344(/17591) words with w2v vectors
Vocab size : 16344


In [3]:
# Whereas this worked first time!
glove = reps.GloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

embeddings = glove.get_embeddings()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Turn that into a DF for me
embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
                              "embeddings": list(embeddings.values())})
embeddings_df.shape

(78403, 2)

In [5]:
embeddings_df.head()

,clean_text,embeddings
0,Hurricane Dorian lashes US as Bahamas counts c...,"[-0.07777254175883196, 0.1746097646540147, 0.3..."
1,Kohistan video murders: Three guilty in 'honou...,"[0.15514222671127229, 0.03370754255320309, 0.0..."
2,MH17 Ukraine plane crash: 'Key witness' releas...,"[0.04240691760001298, -0.001538201091107351, 0..."
3,Article 70: The weddings 'ruined' by Kashmir's...,"[-0.18131744058377358, -0.10373363678670464, -..."
4,Syria war: Turkey warns Europe of new migrant ...,"[-0.11685858725949366, 0.1510065238128605, 0.2..."


## 3. Cluster Text

This is the part where the pipelines get a little more experimental

In [6]:
embeddings_array = np.vstack(embeddings_df['embeddings'])

In [7]:
embeddings_array.shape

(78403, 100)

In [8]:
# First, PCA the data
pca = PCA(n_components=30, svd_solver='full')

embeddings_pca = pca.fit_transform(embeddings_array)

In [9]:
embeddings_pca.shape

(78403, 30)

In [10]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

[0.11450513 0.08612553 0.07398933 0.05283158 0.04327281 0.0328331
 0.03056432 0.02584923 0.02450891 0.02126032 0.02077316 0.01868396
 0.01709245 0.01514302 0.01480605 0.0144647  0.01388297 0.01203326
 0.01145093 0.01102217 0.01063196 0.01011868 0.00961605 0.00924258
 0.00904524 0.00863074 0.00809733 0.00796561 0.00781669 0.00769346]
[136.30956952 118.21697023 109.57169974  92.58926965  83.79563923
  72.9910845   70.42408908  64.76460297  63.06318254  58.7352393
  58.05841053  55.06153306  52.66426291  49.57013816  49.01550837
  48.44718757  47.46299126  44.18812111  43.10565929  42.29093844
  41.53560183  40.52058777  39.50137058  38.72670074  38.31103897
  37.4229441   36.24805653  35.95203148  35.61438133  35.33252528]


In [11]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=1)
clusterer.fit(embeddings_pca)
pd.unique(clusterer.labels_)

array([ 3, -1,  0,  1,  2], dtype=int64)

In [12]:
len(pd.unique(clusterer.labels_))

5

In [13]:
Counter(clusterer.labels_)

Counter({3: 75005, -1: 3377, 0: 6, 1: 5, 2: 10})